## Bayesian Sets Recommender System - Pypi

In [ ]:
# Algorithm Implementation: https://github.com/MaLL-UFSCar/bayessets
# Research Paper: 

In [1]:
!pip install git+https://github.com/MaLL-UFSCar/bayessets.git

  Cloning https://github.com/MaLL-UFSCar/bayessets.git to /tmp/pip-req-build-g7zy8t7z
  Running command git clone -q https://github.com/MaLL-UFSCar/bayessets.git /tmp/pip-req-build-g7zy8t7z
  Created wheel for BayesSets: filename=BayesSets-0.2.1-cp36-none-any.whl size=3891 sha256=03db316e6d8d59cd610c9575d2859c6263905749cda4f7e6ea8fb63582b67e20
  Stored in directory: /tmp/pip-ephem-wheel-cache-fgrtccf6/wheels/17/99/fe/bc3c266fdef546aaf825421da93f0826c5eb36802c60c7247e
Successfully built BayesSets


In [1]:
!pip show -f BayesSets

In [2]:
import numpy
import numpy as np
import pandas as pd
from scipy import sparse
import itertools

In [3]:
# check working directory
import os
os.getcwd()

'/home/antrived/Redhat_Work/PycharmProjects/10-Bayesian-Sets-Recom-Pypi'

### Data Loading & Pre-processing

In [4]:
# place 'package-to-id-dict-without-trans.json' and 'manifest-to-id-without-trans.pickle'
    # in wd from 'aagshah-pypi-insights' bucket

In [5]:
def cal_sparsity(array):
    num_total = total_elems(array)
    num_non_zero = zero_elems(array, num_total)
    sparsity = num_non_zero/num_total
    print("Sparsity of matrix is = {}".format(sparsity))
    return sparsity

def zero_elems(array, num_total):
    non_zero = np.count_nonzero(array)
    return num_total-non_zero

def total_elems(array):
    shape = array.shape
    return shape[0]*shape[1]

In [6]:
import json
import pickle

In [7]:
with open('package-to-id-dict-without-trans.json', 'r') as f:
    pack_to_id = json.load(f)

In [8]:
with open('manifest-to-id-without-trans.pickle', 'rb') as f:
    man_to_id = pickle.load(f)

In [9]:
pack_to_id

{'flask': 0,
 'requests': 1,
 'django': 2,
 'simplejson': 3,
 'whitenoise': 4,
 'psycopg2': 5,
 'django-crispy-forms': 6,
 'flake8': 7,
 'gunicorn': 8,
 'dj-database-url': 9,
 'django-bootstrap-form': 10,
 'pytest': 11,
 'click': 12,
 'pandas': 13,
 'schedule': 14,
 'dpath': 15,
 'jsonpointer': 16,
 'pyexcel-io': 17,
 'texttable': 18,
 'ricecooker': 19,
 'django-coverage': 20,
 'coverage': 21,
 'django-debug-toolbar': 22,
 'wsgiref': 23,
 'django-ckeditor': 24,
 'pillow': 25,
 'markdown': 26,
 'python-markdown-math': 27,
 'pygments': 28,
 'olefile': 29,
 'scipy': 30,
 'wheel': 31,
 'coveralls': 32,
 'xenapi': 33,
 'pylint': 34,
 'mock': 35,
 'oslo-db': 36,
 'osprofiler': 37,
 'keystoneauth1': 38,
 'weakrefmethod': 39,
 'pecan': 40,
 'oslo-versionedobjects': 41,
 'oslo-policy': 42,
 'os-traits': 43,
 'duo-client': 44,
 'yapf': 45,
 'enum34': 46,
 'configparser': 47,
 'flask-sqlalchemy': 48,
 'flask-wtf': 49,
 'beautifulsoup4': 50,
 'flask-bootstrap': 51,
 'argparse': 52,
 'pg8000': 53,


In [10]:
man_to_id

{frozenset({'django', 'flask', 'requests', 'simplejson'}): 0,
 frozenset({'dj-database-url',
            'django',
            'django-crispy-forms',
            'flake8',
            'gunicorn',
            'psycopg2',
            'whitenoise'}): 1,
 frozenset({'django', 'django-bootstrap-form', 'psycopg2'}): 2,
 frozenset({'click', 'pytest'}): 3,
 frozenset({'pandas', 'requests', 'schedule'}): 4,
 frozenset({'dpath', 'jsonpointer'}): 5,
 frozenset({'pyexcel-io', 'texttable'}): 6,
 frozenset({'ricecooker'}): 7,
 frozenset({'coverage',
            'django-ckeditor',
            'django-coverage',
            'django-debug-toolbar',
            'psycopg2',
            'wsgiref'}): 8,
 frozenset({'django',
            'markdown',
            'olefile',
            'pillow',
            'pygments',
            'python-markdown-math'}): 9,
 frozenset({'scipy', 'wheel'}): 10,
 frozenset({'coveralls', 'mock', 'pylint', 'xenapi'}): 11,
 frozenset({'keystoneauth1',
            'os-traits',
   

In [11]:
man_to_id.get(frozenset(['django']))

16101

In [12]:
users = len(man_to_id)
print(users) # 66018
items = len(pack_to_id)
print(items) # 18796

66018
18796


In [13]:
# initialize user-item matrix
rating_matrix = np.zeros((users, items))

In [14]:
# populate user-item matrix
for item_list, user in man_to_id.items():
    for item in item_list:
        rating_matrix[user][pack_to_id.get(item)] = 1
# view matrix
print(rating_matrix.shape[0])
print(rating_matrix.shape[1])
rating_matrix[:3] # gets first 3 rows

66018
18796


array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [15]:
cal_sparsity(rating_matrix)
print(sum(sum(rating_matrix))) # no. of ratings = 484717

Sparsity of matrix is = 0.9996093746247605
484717.0


In [16]:
sparse_rating_matrix = sparse.csr_matrix(rating_matrix)
print(sparse_rating_matrix.shape[0]) # 66018
print(sparse_rating_matrix.shape[1]) # 18796

66018
18796


In [17]:
sparse_rating_matrix = sparse_rating_matrix.transpose()
print(sparse_rating_matrix.shape[0]) 
print(sparse_rating_matrix.shape[1])

18796
66018


In [18]:
sparse_rating_matrix[:10]

<10x66018 sparse matrix of type '<class 'numpy.float64'>'
	with 48037 stored elements in Compressed Sparse Column format>

### Training

In [19]:
import bayessets
model = bayessets.BernoulliBayesianSet(sparse_rating_matrix)

In [20]:
with open('Bayesian_Sets.pkl', 'wb') as f:
    pickle.dump(model, f)

In [21]:
with open('Bayesian_Sets.pkl', 'rb') as f:
    model = pickle.load(f)

### Scoring

In [22]:
import random
import math
from math import ceil

def get_packages_from_id(package_ids):
    package_list = list()
    for i in package_ids:
        package = id_to_pack.get(str(i))
        package_list.append(package)
    return package_list

def map_input_to_package_ids(input_stack):
    package_id_list = list()
    for package in input_stack:
        package_id = pack_to_id.get(package)
        if package_id is not None:
            package_id_list.append(package_id)
    return package_id_list

# calculate probability of recommendatios
def calculate_recomm_prob(recommendations, scores):
#     normalized_scores=(scores-scores.min())/(scores.max()-scores.min())
    scores_log = [math.log(i) if i > 0 else 0 for i in scores]
    scores_log = np.array(scores_log)
    normalized_scores=(scores_log-scores_log.min())/(scores_log.max()-scores_log.min())
    Prob = []
    for index in recommendations:
        Prob.append(int(100*normalized_scores[index]))
    return Prob

In [23]:
with open('id-to-package-dict-without-trans.json', 'r') as f:
    id_to_pack = json.load(f)

In [32]:
input_stack = ['flask', 'flask-sockets', 'gevent', 'gevent-websocket']

In [27]:
# input your user_stack here
str1 = str(
"""
Flask==0.10.1
Flask-Sockets==0.2.0
gevent==1.0.2
gevent-websocket==0.9.5
greenlet==0.4.9
gunicorn==19.4.5
itsdangerous==0.24
Jinja2==2.8
MarkupSafe==0.23
redis==2.10.5
Werkzeug==0.11.3
"""
)

In [28]:
import re

def get_pk(s1):
    if "==" in s1:
        return (s1.split("==")[0].strip())
    elif ">=" in s1:
        return (s1.split(">=")[0].strip())
    elif ">" in s1:
        return (s1.split(">")[0].strip())
    else:
        return (s1.strip())

str1 = str1.split("\n")
str1 = str1[1:len(str1)-1]
input_stack = [(re.sub('[^a-zA-Z0-9-_>=]', '', x)).lower() for x in str1 ]
input_stack = [x.replace("_","-") for x in input_stack]
input_stack = input_stack[0:len(input_stack)]
input_stack = [get_pk(x) for x in input_stack]
print(input_stack)

['flask', 'flask-sockets', 'gevent', 'gevent-websocket', 'greenlet', 'gunicorn', 'itsdangerous', 'jinja2', 'markupsafe', 'redis', 'werkzeug']


In [29]:
input_stack = map_input_to_package_ids(input_stack)
print(input_stack)
# QC
print(get_packages_from_id(input_stack))

[0, 2373, 561, 751, 503, 8, 1296, 301, 840, 59, 492]
['flask', 'flask-sockets', 'gevent', 'gevent-websocket', 'greenlet', 'gunicorn', 'itsdangerous', 'jinja2', 'markupsafe', 'redis', 'werkzeug']


In [30]:
# Scoring
scores = model.query(list(input_stack))
ranking = numpy.argsort(scores)[::-1]
top30 = ranking[:30]
recommendations = numpy.array(list(itertools.compress(top30,
    [i not in input_stack for i in top30])))
print(recommendations)
print(get_packages_from_id(recommendations))
print(calculate_recomm_prob(recommendations, scores))

[  5   9  23   1   4 267 136  26  92 326 325  64 275 295  52 220  55 152
  69   3  61  93  85]
['psycopg2', 'dj-database-url', 'wsgiref', 'requests', 'whitenoise', 'dj-static', 'python-dateutil', 'markdown', 'flask-script', 'django-toolbelt', 'static3', 'pymongo', 'flask-restful', 'django-extensions', 'argparse', 'sqlalchemy', 'pyyaml', 'mako', 'ipython', 'simplejson', 'pycrypto', 'flask-login', 'pytz']
[89, 84, 83, 79, 77, 74, 72, 67, 66, 65, 64, 64, 64, 63, 63, 62, 61, 60, 59, 57, 57, 55, 50]


In [31]:
recom = get_packages_from_id(recommendations)
probs = calculate_recomm_prob(recommendations, scores)
recom_prob = []
for i, prob in enumerate(probs):
    if prob >= 10:
#         recom_prob.append(str("{a}({b}%)".format( a=recom[i],b=probs[i] )))
        recom_prob.append(str("{a}".format( a=recom[i] )))

recom_prob

['psycopg2',
 'dj-database-url',
 'wsgiref',
 'requests',
 'whitenoise',
 'dj-static',
 'python-dateutil',
 'markdown',
 'flask-script',
 'django-toolbelt',
 'static3',
 'pymongo',
 'flask-restful',
 'django-extensions',
 'argparse',
 'sqlalchemy',
 'pyyaml',
 'mako',
 'ipython',
 'simplejson',
 'pycrypto',
 'flask-login',
 'pytz']

###########################################################
######### Class Based Implementation (for in-production) ##########
###########################################################

In [118]:
# !pip3 install git+https://github.com/fabric8-analytics/fabric8-analytics-rudra#egg=rudra

In [ ]:
# !pip install git+https://github.com/MaLL-UFSCar/bayessets.git

In [1]:
# import os
import boto3
import math
import numpy as np
import pandas as pd
# from scipy import sparse
import itertools
import json
import pickle
# import bayessets

In [2]:
# bucket_name='antrived-hpf-pypi-insights',
# s3_client = boto3.client(
#     's3',
# #     bucket_name='antrived-hpf-pypi-insights',
#     aws_access_key_id=os.environ.get('AWS_S3_ACCESS_KEY_ID', ''),                 
#     aws_secret_access_key=os.environ.get('AWS_S3_SECRET_ACCESS_KEY', ''))

In [26]:
## Scoring Service

class BSScoring:
    
    def __init__(self, num_recom=None, data_store=None):
        self.pack_to_id = self._load_package_to_id_map()
        self.man_to_id = self._load_manifest_to_id_map()
        self.id_to_pack = self._load_id_to_package_map()
        self.model = self._load_model()
        self.num_recom = num_recom
    
    def _load_package_to_id_map(self):
        """Load package to id map."""
        with open('package-to-id-dict-without-trans.json', 'r') as f:
            pack_to_id = json.load(f)
        return pack_to_id
    
    def _load_manifest_to_id_map(self):
        with open('manifest-to-id-without-trans.pickle', 'rb') as f:
            man_to_id = pickle.load(f)
        return man_to_id
    
    def _load_id_to_package_map(self):
        with open('id-to-package-dict-without-trans.json', 'r') as f:
            id_to_pack = json.load(f)
        return id_to_pack
    
    def _load_model(self):
        with open('Bayesian_Sets.pkl', 'rb') as f:
            model = pickle.load(f)
        return model
    
    def _get_packages_from_id(self, package_ids):
        package_list = list()
        for i in package_ids:
            package = self.id_to_pack.get(str(i))
            package_list.append(package)
        return package_list
    
    def _get_ids_from_packages(self, input_stack):
        package_id_list = list()
        for package in input_stack:
            package_id = self.pack_to_id.get(package)
            if package_id is not None:
                package_id_list.append(package_id)
        return package_id_list
    
    def _calculate_recomm_prob(self, recommendations, scores):
        scores_log = [math.log(i) if i > 0 else 0 for i in scores]
        scores_log = np.array(scores_log)
        normalized_scores=(scores_log-scores_log.min())/(scores_log.max()-scores_log.min())
        Prob = []
        for index in recommendations:
            Prob.append(int(100*normalized_scores[index]))
        return Prob
    
    def predict(self, input_stack):
        input_stack = self._get_ids_from_packages(input_stack)
        scores = self.model.query(list(input_stack))
        ranking = np.argsort(scores)[::-1]
        top30 = ranking[:30]
        recommendations = np.array(list(itertools.compress(top30,
            [i not in input_stack for i in top30])))
        
        recom = self._get_packages_from_id(recommendations)
        probs = self._calculate_recomm_prob(recommendations, scores)
        recom_by_prob = []
        for i, prob in enumerate(probs):
            if prob >= 10:
        #         recom_prob.append(str("{a}({b}%)".format( a=recom[i],b=probs[i] )))
                recom_by_prob.append(str("{a}".format( a=recom[i] )))
            
        recom_by_prob = [x for i, x in enumerate(recom_by_prob) if i<self.num_recom]
            
        companion_packages = []
        for i, rec in enumerate(recom_by_prob):
            key_val = {}
            key_val['package_name'] = rec
            key_val['cooccurrence_probability'] = str(probs[i])
            key_val['topic_list'] = []
            companion_packages.append(key_val)
            
        missing_packages = []

        return companion_packages, missing_packages

In [43]:
input_stack = [
'alembic',
'bcrypt',
'click',
'dnspython',
]

In [44]:
temp = BSScoring(num_recom=10,data_store=2)

In [45]:
temp.pack_to_id
temp.man_to_id
temp.id_to_pack
temp.model
# input_stack = temp._get_ids_from_packages(input_stack)
# print(input_stack)
# print( temp._get_packages_from_id(input_stack) )

In [46]:
limit = 5
companion_packages, missing_packages = temp.predict(input_stack)
companion_packages = [x for i, x in enumerate(companion_packages) if i < limit]

In [47]:
## Recommendation Service
ECOSYSTEM = 'pypi'


response_json = []
response_json.append({
    "missing_packages": missing_packages,
    "companion_packages": companion_packages,
    "ecosystem": ECOSYSTEM
})

response_json

[{'missing_packages': [],
  'companion_packages': [{'package_name': 'flask-bcrypt',
    'cooccurrence_probability': '59',
    'topic_list': []},
   {'package_name': 'python-editor',
    'cooccurrence_probability': '58',
    'topic_list': []},
   {'package_name': 'cffi',
    'cooccurrence_probability': '50',
    'topic_list': []},
   {'package_name': 'mako',
    'cooccurrence_probability': '47',
    'topic_list': []}],
  'ecosystem': 'pypi'}]

In [215]:
# Scoring
scores = model.query(list(input_stack))
ranking = numpy.argsort(scores)[::-1]
top30 = ranking[:30]
recommendations = numpy.array(list(itertools.compress(top30,
    [i not in input_stack for i in top30])))
print(recommendations)
print(get_packages_from_id(recommendations))
print(calculate_recomm_prob(recommendations, scores))

[ 933  583  134  152  580  348 2398  840 3190 4353 4352 4355 4080 2889
 3225 1214 2064 3536 2603  206 1084  581  541 4354 2890 4279]
['flask-bcrypt', 'python-editor', 'cffi', 'mako', 'sqlalchemy-utils', 'passlib', 'zope-sqlalchemy', 'markupsafe', 'pyramid-tm', 'jaydebeapi', 'hive-thrift-py', 'pysmbclient', 'colander', 'pyhive', 'chartkick', 'sphinx-pypi-upload', 'ldap3', 'filechunkio', 'wtforms-components', 'croniter', 'flask-admin', 'statsd', 'tabulate', 'pydruid', 'impyla', 'paginate']
[59, 58, 50, 47, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [229]:
recom = get_packages_from_id(recommendations)
probs = calculate_recomm_prob(recommendations, scores)
recom_prob = []
for i, prob in enumerate(probs):
    if prob >= 10:
#         recom_prob.append(str("{a}({b}%)".format( a=recom[i],b=probs[i] )))
        recom_prob.append(str("{a}".format( a=recom[i] )))

recom_prob

['flask-bcrypt', 'python-editor', 'cffi', 'mako']

In [225]:
recom_by_prob = recom_prob.copy()

In [233]:
companions = []
for i, rec in enumerate(recom_by_prob):
    key_val = {}
    key_val['package_name'] = rec
    key_val['cooccurrence_probability'] = str(probs[i])
    key_val['topic_list'] = []
    companions.append(key_val)

In [234]:
companions

[{'package_name': 'flask-bcrypt',
  'cooccurrence_probability': '59',
  'topic_list': []},
 {'package_name': 'python-editor',
  'cooccurrence_probability': '58',
  'topic_list': []},
 {'package_name': 'cffi', 'cooccurrence_probability': '50', 'topic_list': []},
 {'package_name': 'mako', 'cooccurrence_probability': '47', 'topic_list': []}]

In [33]:
probs

[89,
 84,
 83,
 79,
 77,
 74,
 72,
 67,
 66,
 65,
 64,
 64,
 64,
 63,
 63,
 62,
 61,
 60,
 59,
 57,
 57,
 55,
 50]